## Naive Autocorrect
1. This is a Naive Auto Correct program.<br>
2. This code is a novice implementation as it is trained on limited data and the prediction does not consider the context.<br>
3. But this code can still predict <font color=#068DA9> the correct dictionary word one or two distances away</font>.<br>
4. I also implemented the minimum edit distance using <font color=#068DA9>Levenshtein distance</font> which uses the concept of <b>Dynamic programming</b><br> 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
def process_words(file_name):
    words=[]
    with open(file_name) as file:
        file_contents = file.read().lower()
        # print(file_contents)
        words=re.findall('\w+',file_contents)
    return words


In [3]:
words=process_words("shakespeare.txt")
vocab=set(words)
print(f"The first ten words in the text are: \n{words[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")


The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


In [4]:
def get_count(words):
    #Finding the count of Each word in text
    dic={}
    dic=Counter(words)
    return dic

In [5]:
word_count_dict = get_count(words)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")


There are 6116 key values pairs
The count for the word 'thee' is 240


In [6]:
def get_probs(word_count_dict):
    #Finding probabilities of the words
    probs={}
    values=list(word_count_dict.values())
    total_words=np.sum(values)
    for word in word_count_dict.keys():
        count=word_count_dict[word]
        # print(type(count))
        probs[word]=count/total_words
    return probs

In [7]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


In [8]:
def delete_letter(word,verbose=True):
    #This function will generate words by deleting the one letter for given word.
    delete_l=[]
    splits_l=[]
    for i in range(len(word)):
        splits_l.append((word[:i],word[i:]))
    # print(splits_l)
    delete_l=[L+R[1:] for L,R in splits_l if R] 
    # print(delete_l)
    if verbose:
        print(
            f"Input word = {word} \nsplit_l = {splits_l} \delete_l = {delete_l}")
    return delete_l

In [9]:
delete_l=delete_letter(word="cans",verbose=True)

Input word = cans 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')] \delete_l = ['ans', 'cns', 'cas', 'can']


In [10]:
def switch_letter(word,verbose=False):
    #This function will  generate words by swaping the adjacent letters of given word.
    def swap(c,i,j):
        c=list(c)
        c[i],c[j]=c[j],c[i]
        return ''.join(c)
    
    switch_l=[]
    splits_l=[]
    len_word=len(word)
    for i in range(0,len_word):
        splits_l.append((word[:i],word[i:]))
    # print(splits_l)
    for L,R in splits_l:
        if len(R)>=2:
            i=len(L)-1
            j=len(L)
            c=swap(L+R,i,j)
            switch_l.append(c)
    if verbose:
        print(
            f"Input word = {word} \nsplit_l = {splits_l} \nswitch_l = {switch_l}")
    return switch_l


In [11]:
switch_l=switch_letter('eta',verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['ate', 'tea']


In [12]:
#test2#
print(
    f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")


Number of outputs of switch_letter('at') is 1


In [13]:
def replace_letter(word,verbose=False):
    #This function will generate words by replacing the one letter of the word with remaining letters from a-z.
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    for i in range(0,len(word)):
        split_l.append((word[:i],word[i:]))
    for L,R in (split_l):
        if R:
            c=R[0]
            for letter in (letters):
                if c!=letter:
                    replace_l.append(L+letter+R[1:])
    replace_l=sorted(list(set(replace_l)))
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    return replace_l

In [14]:
replace_l=(replace_letter('can',verbose=True))

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [15]:
def insert_letter(word,verbose=True):
    #This function will words by inserting the one letter in the  given word by letters in a-z.#
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    for i in range(len(word)+1):
        split_l.append((word[:i],word[i:]))
    for L,R in (split_l):
        for letter in (letters):
            insert_l.append(L+letter+R)
    insert_l=sorted(list((insert_l)))
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \ninsert_l {insert_l}")   
    return insert_l


In [16]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")


Input word = at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l ['aat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat']
Number of strings output by insert_letter('at') is 78


In [17]:
def edit_one_letter(word,allow_switches=True):
    edit_one_set=set()
    edit_one_set.update(delete_letter(word, verbose=False))
    if allow_switches:
        edit_one_set.update(switch_letter(word,verbose=False))
    edit_one_set.update(insert_letter(word,verbose=False))
    edit_one_set.update(replace_letter(word,verbose=False))
    return edit_one_set

In [18]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(
    f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(
    f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")


input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


In [19]:
def edit_two_letters(word, allow_switches=True):
    edit_two_set=set()
    edit_one_words_set=edit_one_letter(word,allow_switches)
    for edited_word in edit_one_words_set:
        edit_one_edited_words=edit_one_letter(edited_word,allow_switches)
        edit_two_set.update(edit_one_edited_words)
    return edit_two_set

In [20]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(
    f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(
    f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")


Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


In [21]:
def get_corrections(word, probs, vocab, n=2, verbose=False):
    #This is the Ultimate Function to get n best words sunggestion for given word
    suggestions=[]
    n_best=[]
    edit_one_set=edit_one_letter(word, allow_switches=True)
    edit_two_set=edit_two_letters(word,allow_switches=True)
    real_one_edit_words=vocab.intersection(edit_one_set)
    real_two_edit_words=vocab.intersection(edit_two_set)
    suggestions=list(real_one_edit_words or real_two_edit_words)
    probs_suggestion=[probs[suggestion] for suggestion in suggestions]
    sorted_probs=np.argsort(probs_suggestion)[-n:]
    for index in sorted_probs:
        word=suggestions[index]
        probability_word=probs[word]
        n_best.append((word,probability_word))
    if verbose: print("suggestions = ", suggestions)
    n_best=reversed(n_best)
    return n_best

    

In [25]:
my_word = 'dye'
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=False)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")
# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")


word 0: eye, probability 0.000914
word 1: die, probability 0.000560
data type of corrections <class 'list_reverseiterator'>


In [36]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    #Finding minimun distance from target to 
    m=len(source)
    n=len(target)
    dp=np.zeros((m+1,n+1),dtype=int)
    for i in range(0,m+1):
        dp[i][0]=i
    for j in range(0,n+1):
        dp[0][j]=j
    for i in range(1,m+1):
        for j in range(1,n+1):
            if(source[i-1]==target[j-1]):
                dp[i][j]=dp[i-1][j-1]
            else:
                insert=ins_cost+dp[i][j-1]
                delete=del_cost+dp[i-1][j]
                replace=rep_cost+dp[i-1][j-1]
                dp[i][j]=min(insert,min(delete,replace))
    med=dp[m][n]
    return dp,med

In [37]:
source = 'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ", min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns=cols)
print(df)


minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [38]:
source = 'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ", min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns=cols)
print(df)


minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3


In [40]:
source = "eer"
targets = edit_one_letter(source,allow_switches = False)  #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, t, min_edits)

In [41]:
source = "eer"
targets = edit_two_letters(source,allow_switches = False) #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

eer eer 0
